In [3]:
from scripts.load_data import extract_labeled_tokens, label_mapping, read_tsv_file

### Sizes of the datasets before and after emerging enitities

In [4]:
# paths to the data
path_train_original = "data/da_news/da_news_train.tsv"
path_dev_original = "data/da_news/da_news_dev.tsv"
path_test_original = "data/da_news/da_news_test.tsv"

path_train_emerging = "data/no_overlap_da_news/da_news_train.tsv"
path_dev_emerging = "data/no_overlap_da_news/da_news_dev.tsv"
path_test_emerging = "data/no_overlap_da_news/da_news_test.tsv"

In [5]:
# create mapping
label2id, id2label = label_mapping(path_train_original)

In [6]:
# read in the datasets
train_data_original = read_tsv_file(path_train_original, label2id)
dev_data_original = read_tsv_file(path_dev_original, label2id)
test_data_original = read_tsv_file(path_test_original, label2id)

train_data_emerging = read_tsv_file(path_train_emerging, label2id)
dev_data_emerging = read_tsv_file(path_dev_emerging, label2id)
test_data_emerging = read_tsv_file(path_test_emerging, label2id)

In [ ]:
print("Before emerging entities")
print('\tSize of training set:', len(train_data_original))
print('\tSize of dev set:', len(dev_data_original))
print('\tSize of test set:', len(test_data_original))

print("\nAfter emerging entities")
print('\tSize of training set:', len(train_data_emerging))
print('\tSize of dev set:', len(dev_data_emerging))
print('\tSize of test set:', len(test_data_emerging))

Before emerging entities
	Size of training set: 4383
	Size of dev set: 564
	Size of test set: 565

After emerging entities
	Size of training set: 4411
	Size of dev set: 549
	Size of test set: 552
